In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *
from helpers import *
from tools import *

In [2]:
# 原本
data_tr = pd.read_csv('data/train.csv')
idx_tr = data_tr.Id

In [3]:
# 代替
# prepare the data
yb1, data_tr_test, idx_tr_test, labels= load_csv_data('data/train.csv', sub_sample=False) # colomn [id] and [prediction] are not included
# use self-defined DataFrame class to simply collect and manipulate the data
labels_dataframe = ['Prediction']+ labels
data_tr_test_dataframe = np.concatenate((yb1.reshape([-1,1]),data_tr_test), axis=1)
dataframe_tr_test=DataFrame(data_tr_test_dataframe, idx_tr_test.tolist(), labels_dataframe)

In [4]:
# 原本
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b

In [5]:
# 代替: 很慢，谨慎运行
# temp = dataframe_tr_test.loc(dataframe_tr_test['DER_mass_MMC']!=-999)
# temp = temp.loc(temp['Prediction']==1)
# DER_mode_s_test = temp.value_counts('DER_mass_MMC').index[0]

# temp = dataframe_tr_test.loc(dataframe_tr_test['DER_mass_MMC']!=-999)
# temp = temp.loc(temp['Prediction']==-1)
# DER_mode_b_test = temp.value_counts('DER_mass_MMC').index[0]

DER_mode_s_test =119.89
DER_mode_b_test =96.819# 多个值有相同个数，所以于原本不同

temp = dataframe_tr_test.loc(dataframe_tr_test['DER_mass_MMC']==-999)
temp = temp.loc(temp['Prediction']==1)
dataframe_tr_test.loc(temp.index)['DER_mass_MMC'] = DER_mode_s_test

temp = dataframe_tr_test.loc(dataframe_tr_test['DER_mass_MMC']==-999)
temp = temp.loc(temp['Prediction']==-1)
dataframe_tr_test.loc(temp.index)['DER_mass_MMC'] = DER_mode_b_test

In [6]:
# 原本
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.000000
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [7]:
# 代替
def missing_rate_test(dataframe):
    features = dataframe.drop(['Prediction'])
    missing_rate = []
    for feature in features.labels:
        data = dataframe.loc(features[feature]==-999)
        missing_rate.append(data[feature].size/features[feature].size)
    missing_rate = np.array(missing_rate).reshape([1,-1]).squeeze()
    missing_rate_labels = dataframe.labels[1:]
    return DataFrame(missing_rate,[0],missing_rate_labels)

df_miss_tr_test = missing_rate_test(dataframe_tr_test)
df_miss_tr_test.values

array([0.      , 0.      , 0.      , 0.      , 0.709828, 0.709828,
       0.709828, 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.709828, 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.399652,
       0.399652, 0.399652, 0.709828, 0.709828, 0.709828, 0.      ])

In [8]:
# 原本
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
    corr1_2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi,'corr0':corr0,'corr1_2':corr1_2}
    
    return df_drop_name

In [9]:
# 代替
def drop_columns_test(df_miss):
    names_70 = np.array(df_miss.labels)[df_miss.values == 0.709828].tolist()
    names_30 = np.array(df_miss.labels)[df_miss.values == 0.399652].tolist()
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
    corr1_2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi,'corr0':corr0,'corr1_2':corr1_2}
    return df_drop_name

In [10]:
drop_col = drop_columns(df_miss_tr)
drop_col

{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num'],
 'corr0': ['DER_mass_MMC', 'DER_pt_tot', 'PRI_tau_pt', 'PRI_lep_pt'],
 'corr1_2': ['DER_mass_MMC',
  'DER_pt_h',
  'DER_sum_pt',
  'PRI_met_sumet',
  'PRI_jet_all_pt',
  'PRI_lep_pt']}

drop_col_test = drop_columns_test(df_miss_tr_test)
drop_col_test

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [11]:
# 原本：
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [12]:
# 替代：
def group_features_by_jet_test(df):
    return {  
        0: df.loc(df['PRI_jet_num'] == 0).copy(),
        1: df.loc(df['PRI_jet_num'] == 1).copy(),
        2: df.loc((df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)).copy(),
    }

In [13]:
feature_tr_grp = group_features_by_jet(data_tr)
feature_tr_grp[0]

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
3,100003,b,143.905,81.417,80.943,0.414,-999.0,-999.0,-999.0,3.310,...,0.060,86.062,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
4,100004,b,175.864,16.915,134.805,16.405,-999.0,-999.0,-999.0,3.891,...,-0.871,53.131,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
8,100008,b,105.594,50.559,100.989,4.288,-999.0,-999.0,-999.0,2.904,...,0.024,129.804,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
10,100010,b,96.415,86.240,79.692,27.201,-999.0,-999.0,-999.0,2.338,...,0.688,250.178,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
13,100013,b,82.488,31.663,64.128,8.232,-999.0,-999.0,-999.0,2.823,...,1.433,163.420,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
14,100014,b,96.415,109.412,14.398,17.323,-999.0,-999.0,-999.0,0.472,...,-1.583,198.616,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
15,100015,s,111.026,32.096,75.271,23.067,-999.0,-999.0,-999.0,3.205,...,2.415,122.176,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
17,100017,s,127.861,50.953,77.267,26.967,-999.0,-999.0,-999.0,2.833,...,-2.975,211.720,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
18,100018,b,96.415,85.186,68.827,5.042,-999.0,-999.0,-999.0,2.116,...,-2.033,151.816,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0
19,100019,b,96.415,88.767,115.058,15.337,-999.0,-999.0,-999.0,2.879,...,-2.577,115.145,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0


In [15]:
feature_tr_grp_test = group_features_by_jet_test(dataframe_tr_test)
feature_tr_grp_test[0].values

array([[  -1.   ,  143.905,   81.417, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,  175.864,   16.915, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,  105.594,   50.559, ..., -999.   , -999.   ,    0.   ],
       ...,
       [  -1.   ,   96.819,   58.179, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,   94.951,   19.362, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,   96.819,   72.756, ..., -999.   , -999.   ,    0.   ]])

In [16]:
corr0

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
Id,1.000000,-0.002778,-0.004709,-0.003248,-0.000686,NaN,NaN,NaN,-0.002736,-0.000686,...,-0.002680,0.003485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_mass_MMC,-0.002778,1.000000,0.082198,0.772553,0.011128,NaN,NaN,NaN,0.519232,0.011128,...,0.001185,0.198762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_mass_transverse_met_lep,-0.004709,0.082198,1.000000,0.082295,-0.020909,NaN,NaN,NaN,-0.303470,-0.020909,...,-0.016301,0.005508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_mass_vis,-0.003248,0.772553,0.082295,1.000000,0.015904,NaN,NaN,NaN,0.658827,0.015904,...,-0.000009,0.240522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_pt_h,-0.000686,0.011128,-0.020909,0.015904,1.000000,NaN,NaN,NaN,-0.034732,1.000000,...,0.007070,0.405886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_deltaeta_jet_jet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_mass_jet_jet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_prodeta_jet_jet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_deltar_tau_lep,-0.002736,0.519232,-0.303470,0.658827,-0.034732,NaN,NaN,NaN,1.000000,-0.034732,...,0.003875,0.049709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DER_pt_tot,-0.000686,0.011128,-0.020909,0.015904,1.000000,NaN,NaN,NaN,-0.034732,1.000000,...,0.007070,0.405886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
p = np.corrcoef(feature_tr_grp_test[0].values.T)
pd.DataFrame(p)

/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.000000,0.051546,-0.413032,0.032609,0.011308,NaN,NaN,NaN,0.220602,0.011308,...,0.001186,0.102839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.051546,1.000000,0.083949,0.772494,0.010760,NaN,NaN,NaN,0.517626,0.010760,...,0.001163,0.198241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.413032,0.083949,1.000000,0.082295,-0.020909,NaN,NaN,NaN,-0.303470,-0.020909,...,-0.016301,0.005508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.032609,0.772494,0.082295,1.000000,0.015904,NaN,NaN,NaN,0.658827,0.015904,...,-0.000009,0.240522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.011308,0.010760,-0.020909,0.015904,1.000000,NaN,NaN,NaN,-0.034732,1.000000,...,0.007070,0.405886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.220602,0.517626,-0.303470,0.658827,-0.034732,NaN,NaN,NaN,1.000000,-0.034732,...,0.003875,0.049709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.011308,0.010760,-0.020909,0.015904,1.000000,NaN,NaN,NaN,-0.034732,1.000000,...,0.007070,0.405886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
feature_tr_grp_test[0].corr()

/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/xia/Files/Semester_1/Machine Learning/ML_Project/Project_1/ML_2018/tools.py:117: RuntimeWarning: invalid value encountered in greater
  corr0_list.append(temp[(corr0[i] > 0.7) | (corr0[i] < -0.7)])
/Users/xia/Files/Semester_1/Machine Learning/ML_Project/Project_1/ML_2018/tools.py:117: RuntimeWarning: invalid value encountered in less
  corr0_list.append(temp[(corr0[i] > 0.7) | (corr0[i] < -0.7)])


[array(['Prediction'], dtype='<U27'),
 array(['DER_mass_MMC', 'DER_mass_vis'], dtype='<U27'),
 array(['DER_mass_transverse_met_lep'], dtype='<U27'),
 array(['DER_mass_MMC', 'DER_mass_vis'], dtype='<U27'),
 array(['DER_pt_h', 'DER_pt_tot'], dtype='<U27'),
 array([], dtype='<U27'),
 array([], dtype='<U27'),
 array([], dtype='<U27'),
 array(['DER_deltar_tau_lep'], dtype='<U27'),
 array(['DER_pt_h', 'DER_pt_tot'], dtype='<U27'),
 array(['DER_sum_pt', 'PRI_tau_pt', 'PRI_lep_pt'], dtype='<U27'),
 array(['DER_pt_ratio_lep_tau'], dtype='<U27'),
 array(['DER_met_phi_centrality'], dtype='<U27'),
 array([], dtype='<U27'),
 array(['DER_sum_pt', 'PRI_tau_pt'], dtype='<U27'),
 array(['PRI_tau_eta'], dtype='<U27'),
 array(['PRI_tau_phi'], dtype='<U27'),
 array(['DER_sum_pt', 'PRI_lep_pt'], dtype='<U27'),
 array(['PRI_lep_eta'], dtype='<U27'),
 array(['PRI_lep_phi'], dtype='<U27'),
 array(['PRI_met'], dtype='<U27'),
 array(['PRI_met_phi'], dtype='<U27'),
 array(['PRI_met_sumet'], dtype='<U27'),
 array

In [14]:
corr0 = feature_tr_grp[0].corr()
# # corr0.DER_mass_MMC.loc(corr0.DER_mass_MMC.values>0.7)
corr0_list = []
for feature in corr0.index:
    corr0_list.append(corr0.loc[(corr0[feature]>0.7) | (corr0[feature]<-0.7)].index)
corr0_list

[Index(['Id'], dtype='object'),
 Index(['DER_mass_MMC', 'DER_mass_vis'], dtype='object'),
 Index(['DER_mass_transverse_met_lep'], dtype='object'),
 Index(['DER_mass_MMC', 'DER_mass_vis'], dtype='object'),
 Index(['DER_pt_h', 'DER_pt_tot'], dtype='object'),
 Index([], dtype='object'),
 Index([], dtype='object'),
 Index([], dtype='object'),
 Index(['DER_deltar_tau_lep'], dtype='object'),
 Index(['DER_pt_h', 'DER_pt_tot'], dtype='object'),
 Index(['DER_sum_pt', 'PRI_tau_pt', 'PRI_lep_pt'], dtype='object'),
 Index(['DER_pt_ratio_lep_tau'], dtype='object'),
 Index(['DER_met_phi_centrality'], dtype='object'),
 Index([], dtype='object'),
 Index(['DER_sum_pt', 'PRI_tau_pt'], dtype='object'),
 Index(['PRI_tau_eta'], dtype='object'),
 Index(['PRI_tau_phi'], dtype='object'),
 Index(['DER_sum_pt', 'PRI_lep_pt'], dtype='object'),
 Index(['PRI_lep_eta'], dtype='object'),
 Index(['PRI_lep_phi'], dtype='object'),
 Index(['PRI_met'], dtype='object'),
 Index(['PRI_met_phi'], dtype='object'),
 Index(['PR

In [ ]:
def corr(self):
    corr0 = np.corrcoef(self.values)
    corr0_list = []
    temp = np.array(self.labels)
    for i in range(corr0.shape[0]):
        corr0_list.append(temp[(corr0[i] > 0.7) | (corr0[i] < -0.7)])
    return corr0_list

In [ ]:
corr(feature_tr_grp_test[0])

In [ ]:
dataframe_tr_test.loc(dataframe_tr_test['PRI_jet_num'] == 0).values

In [ ]:
# 原本
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi']+drop_col['corr1_2'])))

In [ ]:
# 替代
feature_tr_grp_test[0] = feature_tr_grp_test[0].drop(list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_tr_grp_test[0] = feature_tr_grp_test[0].drop('PRI_jet_all_pt')# all 0s
feature_tr_grp_test[1] = feature_tr_grp_test[1].drop(list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_tr_grp_test[2] = feature_tr_grp_test[2].drop(list(set(drop_col['names_phi']+drop_col['corr1_2'])))

In [ ]:
feature_tr_grp[0]

In [ ]:
feature_tr_grp_test[0].values

In [ ]:
feature_tr_grp_test[0].labels

## 以下没改, 问题不大，留待来日（吐了）

In [ ]:
k_fold = 10
seed = 1
lambdas = np.logspace(-10, 2, 20) 

for lambda_ in lambdas:
    x = feature_tr_grp[1].drop(columns = ['Id','Prediction']).values
    x = build_polynomial_features(x,10)
    x_tr0,_,_ = standardize(x)
    y_tr0 = feature_tr_grp[1].Prediction.replace(['s','b'],[1,-1]).values
    w0,acc_tr0,acc_te0 = cv_loop(y_tr0, x_tr0,k_fold,seed,ridge_regression,lambda_=lambda_)
    print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr0, acc_te0))

In [ ]:
# ridge_regression using cv
w = []
acc_tr = []
acc_te = []
lambda_ = 1e-9#0.0001
k_fold = 10
seed = 10

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = build_polynomial_features(x,10)
    x_tr,_,_ = standardize(x)
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
    w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp)
    acc_te.append(acc_te_tmp)

In [ ]:
acc_tr # degree = 10 lambda = 1e-9

In [ ]:
acc_te

In [ ]:
acc_tr # degree = 10 lambda = 0.0001

In [ ]:
acc_te

In [ ]:
acc_tr # degree = 3 lambda = 0.0001

In [ ]:
acc_te

In [ ]:
k_fold = 10
seed = 10
max_iters = 200
gammas = [0.001,0.01,0.1,0.5,1,1.5,2]
gamma = 0.001

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = build_polynomial_features(x,2)
    x_tr,_,_ = standardize(x)
    x_tr = np.hstack((np.ones((x_tr.shape[0], 1)), x_tr))
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values

    loss,w = logistic_regression_GD(y_tr, x_tr, max_iters, gamma = 0.001)
    y_tr = y_tr.reshape(len(y_tr),1)
    y_tr_pred = predict_labels(w, x_tr)
    acc_tr = compute_accuracy(y_tr_pred,y_tr)
    print("Current gamma={i}, loss={l}, acc = {a}".format(i=gamma, l=loss,a = acc_tr))



# k_indices = build_k_indices(y_tr, k_fold = 4, seed = 10)
# k = 0
# test_idx = k_indices[k]
# train_idx = list(set(np.arange(0,len(y_tr)))-set(k_indices[k]))
# [x_train,y_train,x_test,y_test] = [x_tr[train_idx], y_tr[train_idx], x_tr[test_idx], y_tr[test_idx]]
# w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,logistic_regression_GD, max_iters=100, gamma=gamma)

In [ ]:
def build_polynomial_features(x, degree):
    temp_dict = {}
    count = 0
    for i in range(x.shape[1]):
        for j in range(i+1,x.shape[1]):
            temp = x[:,i] * x[:,j]
            temp_dict[count] = [temp]
            count += 1
    poly_length = x.shape[1] * (degree + 1) + count# + 1
    poly = np.zeros(shape = (x.shape[0], poly_length))
    for deg in range(1,degree+1):
        for i in range(x.shape[1]):
            poly[:,i + (deg-1) * x.shape[1]] = np.power(x[:,i],deg)
    for i in range(count):
        poly[:, x.shape[1] * degree + i] = temp_dict[i][0]
    for i in range(x.shape[1]):
        poly[:,i + x.shape[1] * degree + count] = np.abs(x[:,i])**0.5
    return poly

In [ ]:
k_indices = build_k_indices(y_tr, k_fold = 1000, seed = 10)
k = 0
test_idx = k_indices[k]
train_idx = list(set(np.arange(0,len(y_tr)))-set(k_indices[k]))
[x_train,y_train,x_test,y_test] = [x_tr[train_idx], y_tr[train_idx], x_tr[test_idx], y_tr[test_idx]]
# w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,logistic_regression_GD, max_iters=100, gamma=gamma)

In [ ]:
test = np.array([[-1,4,9]])
test

In [ ]:
build_polynomial_features(test, 2)

In [ ]:
x = feature_tr_grp[2].drop(columns = ['Id','Prediction']).values
x_tr = build_polynomial_features(x,3)
